In [1]:
import pandas as pd
import pandas
import seaborn as sns
import pyodbc
from gensim import corpora, models, similarities
import nltk
import binascii
import logging, gensim, bz2
from nltk.tag import pos_tag
import logging
import re
import tempfile
import tarfile, zipfile
from gensim.utils import tokenize
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import numpy as np



/Users/zzhao/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
df = pd.read_csv("/Users/zzhao/Documents/Anthony_code_and_review/a894359.csv")

df["topics_list"] = df["what_topics_is_this_review_about"].str.split("\n")

topics = df["topics_list"].dropna()

import itertools
attributes = pd.DataFrame(topics.apply(lambda x: dict(zip(x, itertools.repeat(1)))).tolist(), index=topics.index)

df = df.join(attributes)

In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
_unit_id,2796,9.29647e+08,1158.82,9.29645e+08,9.29646e+08,9.29647e+08,9.29648e+08,9.29649e+08
_golden,2796,0,0,False,0,0,0,False
_trusted_judgments,2796,3.92704,0.914036,3,3,4,5,7
industry,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
normalized_rating,2796,0.618723,0.392908,0,0.2,0.8,1,1
what_topics_is_this_review_about_gold,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
administrative_process,13,1,0,1,1,1,1,1
bedside_manner,1557,1,0,1,1,1,1,1
care_coordination,15,1,0,1,1,1,1,1
cleanliness,13,1,0,1,1,1,1,1


In [42]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

df = df[pd.notnull(df['topics_list'])]

text_clf = Pipeline([('vect', CountVectorizer(stop_words="english")),
                      ('tfidf', TfidfTransformer()),
                      ('clf', OneVsRestClassifier(DecisionTreeClassifier()))
 ])

from sklearn.grid_search import GridSearchCV
parameters = {'vect__ngram_range': [(1,1),(1, 2),(1,3)],
               'tfidf__use_idf': (True, False),
              'tfidf__norm': ('l1', 'l2'),
              'tfidf__smooth_idf': (True, False),
              'tfidf__sublinear_tf': (True, False),
              'vect__min_df': (.05,.1),
              'vect__max_df': (0.5, 0.75, 1.0),
            'vect__max_features': (None, 5000, 10000, 50000),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, verbose=1)

from sklearn.preprocessing import MultiLabelBinarizer
binarizer = MultiLabelBinarizer()
train_target = binarizer.fit_transform(df.topics_list)


#text_clf = gs_clf.fit(train_data, train_target)
gs_clf.fit(df.comment, train_target)

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   35.8s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   58.6s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed:  4.6min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
     ...it=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 1), (1, 2), (1, 3)], 'tfidf__smooth_idf': (True, False), 'tfidf__use_idf': (True, False), 'tfidf__sublinear_tf': (True, False), 'tfidf__norm': ('l1', 'l2'), 'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (None, 5000, 10000, 50000), 'vect__min_df': (0.05, 0.1)},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [39]:
df.shape

(978, 23)